### Classic:
In Riddler City, all the streets are currently two-way streets. But in an effort to make the metropolis friendlier for pedestrians and cyclists, the mayor has decreed that all streets should be one-way. Meanwhile, the civil engineer overseeing this transition is not particularly invested in the project and will be randomly assigning every block of each street a random direction.

For your daily commute to work, you drive a car two blocks east and two blocks south, as shown in the diagram below. What is the probability that, after each block is randomly assigned a one-way direction, there will still be a way for you to commute to work while staying within this two-by-two block region (i.e., sticking to the 12 streets you see in the diagram)? 

#### Solution:

- Convert to a graph
- Edges have direction
- Traversal from upper-left vertex to bottom-right vertex 

#### Graph Vertex:

```
0 1 2 

3 4 5

6 7 8

```

#### Combinations of Edges:

Helpful to store as a dictionary. But to get randomness of edges it makes more sense to store as tuples, apply randomness, and then convert to graph. 

#### Solving

- Direction is determined by randomly flipping, but if we have (0,1) we go 0 -> 1. (1, 0) means 1 -> 0. 
- Goal is to check if its possible to go from start to end with current layout

#### Example below: 

- Can use recursion. Leaning on my AoC solution: https://github.com/dwanneruchi/AdventOfCode/blob/c0be3a87cace0228eb6852f33a611298220ed526/2020/day07/day07solution_part2.ipynb

In [2]:
l_tuples = [(0,1), (0,3),
            (1,2), (1,4),
            (2,5),
            (3,4), (3,6),
            (4,5), (4,7),
            (5,8),
            (6,7),
            (7,8)
           ]

In [3]:
from collections import defaultdict
routes = defaultdict( list )

for k,v in l_tuples:
    routes[k].append(v)

In [4]:
routes

defaultdict(list,
            {0: [1, 3],
             1: [2, 4],
             2: [5],
             3: [4, 6],
             4: [5, 7],
             5: [8],
             6: [7],
             7: [8]})

In [5]:
def graphSolver(d,k):
    """Iterate over values in key, adding all edges (needs to be cleaned up)"""
    verts = list(d[k])
    
    # iterate over each vert
    for k in verts:
        verts = verts + graphSolver(d, k)
    else:
        pass
    return verts

In [6]:
all_paths = graphSolver(routes, 0)
print(all_paths)

if 8 in all_paths:
    print(True)

[1, 3, 2, 4, 5, 8, 5, 7, 8, 8, 4, 6, 5, 7, 8, 8, 7, 8]
True


In [7]:
# check an impossible example now 
# 0 - 1 exists, but then 1 is flipped to go backwards
# 0 - 3 no longer exists (only goes up)

l_tuples = [(0,1), (3,0),
            (2,1), (4,1),
            (2,5),
            (3,4), (3,6),
            (4,5), (4,7),
            (5,8),
            (6,7),
            (7,8)
           ]


routes = defaultdict( list )

for k,v in l_tuples:
    routes[k].append(v)

# trapped at 1
all_paths = graphSolver(routes, 0)
print(all_paths)

if 8 in all_paths:
    print(True)

[1]


### Error to Solve:

- I can get stuck in an infinite loop. How to avoid? 

- Right now it frustratingly breaks everything...so may avoid recursive func

- Graph Traversal likely makes sense here
    - https://www.python.org/doc/essays/graphs/

In [27]:
# check an infinite loop example now 
# 0 - 1 , 1 -4 , 4 - 3, 3 - 0

# each tuple represents a graph arc
l_tuples = [(0,1), (3,0),
            (2,1), (1,4),
            (2,5),
            (4, 3), (6,3),
            (5,4), (7,4),
            (5,8),
            (6,7),
            (7,8)
           ]


# build a default dict 
routes = defaultdict( list )

# store each arc into a dictionary 
for k,v in l_tuples:
    routes[k].append(v)
    
    
routes

defaultdict(list,
            {0: [1],
             3: [0],
             2: [1, 5],
             1: [4],
             4: [3],
             6: [3, 7],
             5: [4, 8],
             7: [4, 8]})

In [28]:
# utilize the path function from documentation
def find_path(graph, start, end, path=[]):
        path = path + [start]
        if start == end:
            return path
        if start not in graph:
            return None
        for node in graph[start]:
            if node not in path:
                newpath = find_path(graph, node, end, path)
                if newpath: return newpath
        return None

In [30]:
assert(find_path(routes, 0, 8) == None)

### Simulation Test: 

Steps: 
- randomly flip edges (they will now shift into arcs)
- convert to dictionary graph
- determine if a viable route from 0 - 8 exists (not None)

In [32]:
from random import randint

def randomRoute(l_tup):
    """Randomly flip edges to generate list of of arcs"""
    out_tup = []
    for edge in l_tup:
        if randint(0,1) == 1:
            edge = (edge[1], edge[0])
        out_tup.append(edge)
    return out_tup

In [33]:
# original list of edges all east / south
l_tuples = [(0,1), (0,3),
            (1,2), (1,4),
            (2,5),
            (3,4), (3,6),
            (4,5), (4,7),
            (5,8),
            (6,7),
            (7,8)
           ]

success = 0
sims = 10
    
for _ in range(sims):

    # randomize route
    out = randomRoute(l_tuples)
    
    # convert to graph 
    routes = defaultdict( list )
    for k,v in out:
        routes[k].append(v)
        
    print(routes)

    # paths
    try:
        all_paths = find_path(routes, 0,8)
        print(all_paths)
    except:
        print("Error")

    #if 8 in all_paths:
    #    print(True)

defaultdict(<class 'list'>, {0: [1], 3: [0], 2: [1], 4: [1, 3, 7], 5: [2, 4, 8], 6: [3], 7: [6], 8: [7]})
None
defaultdict(<class 'list'>, {1: [0, 2], 0: [3], 4: [1, 7], 5: [2, 4], 3: [4, 6], 8: [5, 7], 7: [6]})
None
defaultdict(<class 'list'>, {0: [1, 3], 1: [2, 4], 5: [2, 4, 8], 3: [4, 6], 7: [4], 6: [7], 8: [7]})
None
defaultdict(<class 'list'>, {0: [1], 3: [0, 4], 1: [2], 4: [1, 7], 5: [2, 4, 8], 6: [3, 7], 8: [7]})
None
defaultdict(<class 'list'>, {1: [0, 4], 0: [3], 2: [1, 5], 3: [4], 6: [3, 7], 4: [5], 7: [4, 8], 8: [5]})
None
defaultdict(<class 'list'>, {1: [0], 3: [0, 4], 2: [1], 4: [1, 5], 5: [2, 8], 6: [3], 7: [4, 6], 8: [7]})
None
defaultdict(<class 'list'>, {1: [0], 0: [3], 2: [1], 4: [1, 5, 7], 5: [2], 3: [4], 6: [3, 7], 8: [5, 7]})
None
defaultdict(<class 'list'>, {0: [1], 3: [0, 6], 1: [2, 4], 5: [2, 8], 4: [3, 5, 7], 6: [7], 8: [7]})
[0, 1, 4, 5, 8]
defaultdict(<class 'list'>, {0: [1, 3], 1: [2], 4: [1], 2: [5], 3: [4, 6], 5: [4], 7: [4, 6, 8], 8: [5]})
None
defaultdic

#### Large Sim: 

In [36]:
# original list of edges all east / south
l_tuples = [(0,1), (0,3),
            (1,2), (1,4),
            (2,5),
            (3,4), (3,6),
            (4,5), (4,7),
            (5,8),
            (6,7),
            (7,8)
           ]

success = 0
sims = 10000
    
for _ in range(sims):

    # randomize route
    out = randomRoute(l_tuples)
    
    # convert to graph 
    routes = defaultdict( list )
    for k,v in out:
        routes[k].append(v)
        
    # paths
    try:
        all_paths = find_path(routes, 0,8)
        if all_paths != None:
            #print(all_paths)
            success += 1
    except:
        print("Error")


print(f"Probability of success: {success/sims:.4f}")

Probability of success: 0.2750


#### Sim Progression: 

- see how the prob of success shifts over time 

In [38]:
# original list of edges all east / south
l_tuples = [(0,1), (0,3),
            (1,2), (1,4),
            (2,5),
            (3,4), (3,6),
            (4,5), (4,7),
            (5,8),
            (6,7),
            (7,8)
           ]

sim_list = [100,1_000,10_000, 100_000, 250_000, 500_000, 1_000_000, 2_000_000]

for sims in sim_list:

    success = 0

    for _ in range(sims):

        # randomize route
        out = randomRoute(l_tuples)

        # convert to graph 
        routes = defaultdict( list )
        for k,v in out:
            routes[k].append(v)

        # paths
        try:
            all_paths = find_path(routes, 0,8)
            if all_paths != None:
                #print(all_paths)
                success += 1
        except:
            print("Error")


    print(f"Probability of success for sim size {sims}: {success/sims:.4f}")

Probability of success for sim size 100: 0.2000
Probability of success for sim size 1000: 0.2590
Probability of success for sim size 10000: 0.2712
Probability of success for sim size 100000: 0.2757
Probability of success for sim size 250000: 0.2779
Probability of success for sim size 500000: 0.2781
Probability of success for sim size 1000000: 0.2772
Probability of success for sim size 2000000: 0.2767


#### Analytical Solution:

- Determine all possible combinations of routes (think of them as + vs -)
- Determine total routes that allow for Start -> Finish 